In [1]:
#| default_exp 15-metadata-label-matrix-augmented-input-and-trie-for-distilbert-with-thresholding-on-amazon-titles

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
#| export
import os, pandas as pd, warnings, torch, pickle, numpy as np
from tqdm.auto import tqdm
from scipy import stats

import xclib.utils.sparse as xs

from xcai.basics import *
from xcai.models.MMM00X import DBT007, DBT008
from xcai.transform import AugmentMetaInputIdsTfm

In [4]:
#| export
os.environ['WANDB_MODE'] = 'disabled'

In [5]:
#| export
dump_dir = '/home/aiscuser/scratch/Projects/xc_nlg/outputs/\
15-metadata-label-matrix-augmented-input-and-trie-for-distilbert-with-thresholding-on-amazon-titles'

In [7]:
block = XCBlock.from_cfg('/home/aiscuser/scratch/datasets', 'data_meta', dset='amazontitles', valid_pct=0.001, 
                         tokenizer='distilbert-base-uncased')

/home/aiscuser/.local/lib/python3.9/site-packages/xclib-0.97-py3.9-linux-x86_64.egg/xclib/data/data_utils.py:263: UserWarning: Header mis-match from inferred shape!
  warnings.warn("Header mis-match from inferred shape!")
/opt/conda/envs/ptca/lib/python3.9/site-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [8]:
block = AugmentMetaInputIdsTfm.apply(block, meta='cat_meta', max_len=512, exclude_sep=True)

In [9]:
block.test.dset.data.data_info['input_ids'] = block.test.dset.data.data_info['input_ids_aug_cat']
block.test.dset.data.data_info['attention_mask'] = block.test.dset.data.data_info['attention_mask_aug_cat']

In [11]:
fname = f'{dump_dir}/data/block.pkl'
os.makedirs(os.path.dirname(fname), exist_ok=True)
with open(f'{dump_dir}/data/block.pkl', 'wb') as f: pickle.dump((block, test_dset), f)

In [ ]:
#| export
fname = f'{dump_dir}/data/block.pkl'
with open(fname, 'rb') as f: block, test_dset = pickle.load(f)

In [30]:
test_dset = block.test.dset.sample(n=2000, seed=50)

# Generation

In [15]:
#| export
mname = f'/home/aiscuser/scratch/Projects/XC-NLG/models/distilbert-base-uncased_RB33-NAR-3+8-2_(mapped)LF-AmazonTitles-1.3M/checkpoint-884000'
model = DBT007.from_pretrained(mname, tn_targ=10_000, ig_tok=0)

Some weights of DBT007 were not initialized from the model checkpoint at /home/aiscuser/scratch/Projects/XC-NLG/models/distilbert-base-uncased_RB33-NAR-3+8-2_(mapped)LF-AmazonTitles-1.3M/checkpoint-884000 and are newly initialized: ['loss_fn.o']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [31]:
#| export
args = XCLearningArguments(
    output_dir=f'{dump_dir}/distilbert-base-uncased_RB33-NAR-3+8-2_(mapped)LF-AmazonTitles-1.3M',
    generation_length_penalty=1.5,
    per_device_eval_batch_size=32,
    evaluation_strategy='steps',
    label_names=['lbl2data_idx'],
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [32]:
#| export
metric = PrecRecl(test_dset.n_lbl, test_dset.data.data_lbl_filterer, prop=block.train.dset.data.data_lbl,
                  pk=10, rk=200, rep_pk=[1, 3, 5, 10], rep_rk=[2, 3, 10, 50, 100, 200])

In [ ]:
metric = PrecRecl(block.n_lbl, block.test.data_lbl_filterer, prop=block.train.dset.data.data_lbl,
                  pk=10, rk=200, rep_pk=[1, 3, 5, 10], rep_rk=[2, 3, 10, 50, 100, 200])

In [33]:
#| export
learn = XCLearner(
    model=model, 
    args=args,
    data_collator=block.collator, 
    compute_metrics=metric,
)

## `meta-augmented trie`

In [19]:
#| export
data_lbl = block.train.dset.data.data_lbl
lbl_lbl = data_lbl.T.dot(data_lbl).tocsr()
lbl_lbl.setdiag(1)

/opt/conda/envs/ptca/lib/python3.9/site-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [20]:
#| export
lbl_toks = block.lbl_info['input_ids']
lbl_info = [o.indices.tolist() for o in lbl_lbl]

trie = Trie.from_list(lbl_toks, lbl_info)

  0%|          | 0/1305265 [00:00<?, ?it/s]

In [22]:
#| export
meta_lbl = block.train.dset.meta.cat_meta.lbl_meta.T.tocsr()

In [23]:
#| export
n_lbl = meta_lbl.getnnz(axis=1)
valid_meta_idx = np.where(np.logical_and(n_lbl>1, n_lbl<100))[0]

In [26]:
#| export
meta_toks = [block.train.dset.meta.cat_meta.meta_info['input_ids'][i] for i in tqdm(valid_meta_idx)]
meta_info = [meta_lbl[i].indices.tolist() for i in tqdm(valid_meta_idx)]

  0%|          | 0/8347 [00:00<?, ?it/s]

  0%|          | 0/8347 [00:00<?, ?it/s]

In [27]:
#| export
trie.update(meta_toks, meta_info)

  0%|          | 0/8347 [00:00<?, ?it/s]

In [36]:
#| export
learn.tbs.trie = trie

In [37]:
#| export
learn.tbs.n_bm = learn.args.generation_num_beams = 20

In [ ]:
#| export
o = learn.predict(test_dset)
print(o.metrics)

#display_metric(o.metrics)

node-0:16187:16187 [0] NCCL INFO Bootstrap : Using eth0:10.13.38.202<0>
node-0:16187:16187 [0] NCCL INFO NET/Plugin : Plugin load (librccl-net.so) returned 2 : librccl-net.so: cannot open shared object file: No such file or directory
node-0:16187:16187 [0] NCCL INFO NET/Plugin : No plugin found, using internal implementation
node-0:16187:16187 [0] NCCL INFO Kernel version: 5.15.0-1042-azure
RCCL version 2.17.1+hip5.7 HEAD:cbbb3d8+

node-0:16187:42701 [1] /long_pathname_so_that_rpms_can_package_the_debug_info/src/extlibs/rccl/build/hipify/src/misc/ibvwrap.cc:222 NCCL WARN Call to ibv_open_device failed

node-0:16187:42701 [1] /long_pathname_so_that_rpms_can_package_the_debug_info/src/extlibs/rccl/build/hipify/src/transport/net_ib.cc:199 NCCL WARN NET/IB : Unable to open device mlx5_0

node-0:16187:42701 [1] /long_pathname_so_that_rpms_can_package_the_debug_info/src/extlibs/rccl/build/hipify/src/misc/ibvwrap.cc:222 NCCL WARN Call to ibv_open_device failed

node-0:16187:42701 [1] /long_pa

/opt/conda/envs/ptca/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


In [ ]:
o = learn.predict(block.test.dset)
print(o.metrics)

In [ ]:
#| export
pred_fname = f'{dump_dir}/distilbert-base-uncased_RB33-NAR-3+8-2_(mapped)LF-AmazonTitles-1.3M/checkpoint-884000/\
predictions/test_lbl_atrie-hlk_n-bm-20_n-smp-500_seed-50.pth'
os.makedirs(os.path.dirname(pred_fname), exist_ok=True)
torch.save(o, pred_fname)

In [ ]:
o

XCPredictionOutput(pred_idx=tensor([ 39908, 214276,  88840,  ..., 266709,  17914, 117626]), pred_ptr=tensor([ 647,  495, 2529,  ...,  340,  430,  143]), pred_score=tensor([-1.0248, -1.0248, -1.0248,  ..., -0.3543, -0.3543, -0.3543]), metrics={'test_loss': 7.055476665496826, 'test_P@1': 0.086, 'test_P@10': 0.02685000000000015, 'test_P@3': 0.054999999999999834, 'test_P@5': 0.04100000000000019, 'test_N@1': 0.0860000029206276, 'test_N@10': 0.10328423976898193, 'test_N@3': 0.09178565442562103, 'test_N@5': 0.09570598602294922, 'test_PSP@1': 0.06632379608438409, 'test_PSP@10': 0.09066957830489748, 'test_PSP@3': 0.07007277305336816, 'test_PSP@5': 0.07571821362865566, 'test_PSN@1': 0.06632379442453384, 'test_PSN@10': 0.08855146169662476, 'test_PSN@3': 0.07530207186937332, 'test_PSN@5': 0.08075478672981262, 'test_R@2': 0.07790799584757387, 'test_R@3': 0.08942541658398437, 'test_R@100': 0.25829073809147635, 'test_R@200': 0.3411634665983813, 'test_R@10': 0.12572872925525738, 'test_R@50': 0.2034787

### __Shortlist & Total recall__

All the numbers reported below are on a sample of 2000 points.

In [ ]:
s = get_tensor_statistics(o.pred_ptr);s

In [ ]:
r, rm = total_recall(o.pred_idx, o.pred_ptr, test_dset.data.data_lbl, test_dset.data.data_lbl_filterer) 
print(f'Total recall : {r*100:.3f}')

In [ ]:
r = get_tensor_statistics(torch.tensor(rm.sum(axis=1)).flatten()*rm.shape[0]*100); r

In [ ]:
data_rec = torch.tensor(rm.sum(axis=1)).flatten()

In [ ]:
import matplotlib.pyplot as plt

set_plot_defaults()
plt.figure(figsize=(7,5))
plt.yscale('log')
idx = torch.where(data_rec > 0)[0]
plt.scatter(data_rec[idx], o.pred_ptr[idx], s=5)

In [ ]:
stats.pearsonr(data_rec, o.pred_ptr), stats.spearmanr(data_rec, o.pred_ptr)

# Representation

In [ ]:
args = XCLearningArguments(
    output_dir=f'{dump_dir}/ngame_v-3-100_(mapped)LF-AmazonTitles-1.3M/',
    index_efs=200,
    representation_num_beams=200,
    generation_length_penalty=1.5,
    per_device_eval_batch_size=16,
    evaluation_strategy='steps',
    label_names=['lbl2data_idx'],
)

In [ ]:
bsz = max(args.per_device_train_batch_size, args.per_device_eval_batch_size)*torch.cuda.device_count()
model = DBT008.from_pretrained('sentence-transformers/msmarco-distilbert-base-v4', bsz=bsz, tn_targ=10_000, ig_tok=0)

Some weights of DBT008 were not initialized from the model checkpoint at sentence-transformers/msmarco-distilbert-base-v4 and are newly initialized: ['loss_fn.u', 'loss_fn.v']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from collections import OrderedDict

mname = f'/home/aiscuser/scratch/Projects/XC/models/NGAME/STransformer/(mapped)LF-AmazonTitles-1.3M/v_3_100/siamese\
/model_network.pkl'
m_state_dict, msd = OrderedDict(), torch.load(mname)
for k in msd:
    o = k.split('.')
    if o[0] == 'encoder': m_state_dict['distilbert.'+'.'.join(o[5:])] = msd[k]
model.load_state_dict(m_state_dict, strict=False)


_IncompatibleKeys(missing_keys=['loss_fn.u', 'loss_fn.v'], unexpected_keys=[])

In [ ]:
metric = PrecRecl(test_dset.n_lbl, test_dset.data.data_lbl_filterer, prop=block.train.dset.data.data_lbl,
                  pk=10, rk=200, rep_pk=[1, 3, 5, 10], rep_rk=[2, 3, 10, 50, 100, 200])

In [ ]:
metric = PrecRecl(block.n_lbl, block.test.data_lbl_filterer, prop=block.train.dset.data.data_lbl,
                  pk=10, rk=200, rep_pk=[1, 3, 5, 10], rep_rk=[2, 3, 10, 50, 100, 200])

In [ ]:
learn = XCLearner(
    model=model, 
    args=args,
    data_collator=block.collator, 
    compute_metrics=metric,
)

/scratch/scai/phd/aiz218323/anaconda3/envs/xc_nlg/lib/python3.9/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


## Reranking

In [ ]:
import torch.nn.functional as F

In [ ]:
data_repr = F.normalize(learn.get_representation(learn.get_test_dataloader(test_dset)), dim=1)

  0%|          | 0/63 [00:00<?, ?it/s]

/scratch/scai/phd/aiz218323/Projects/xcai/xcai/losses.py:21: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at ../aten/src/ATen/SparseCsrTensorImpl.cpp:53.)
  return torch.sparse_csr_tensor(data_ptr, data_idx, scores, device=data_ptr.device)


In [ ]:
lbl_repr = F.normalize(learn.get_representation(learn.get_test_dataloader(test_dset.lbl_dset)), dim=1)

  0%|          | 0/9761 [00:00<?, ?it/s]

In [ ]:
pred = torch.load(f'{dump_dir}/distilbert-base-uncased_RB33-NAR-3+8-2_(mapped)LF-AmazonTitles-1.3M/checkpoint-884000/\
predictions/test_lbl_atrie-hlk_n-bm-20_n-smp-500_seed-50.pth')

In [ ]:
from tqdm.auto import tqdm
sc,p = [],0
for q,d in tqdm(zip(pred.pred_ptr.cumsum(0),data_repr), total=data_repr.shape[0]):
    sc.append(d@lbl_repr[pred.pred_idx[p:q]].T);p=q
sc = torch.cat(sc)

  0%|          | 0/2000 [00:00<?, ?it/s]

In [ ]:
o = {
    'pred_idx': pred.pred_idx,
    'pred_ptr': pred.pred_ptr,
    'pred_score': pred.pred_score,
}
o['targ_idx'] = torch.tensor(test_dset.data.data_lbl.indices, dtype=torch.long)
o['targ_ptr'] = torch.tensor([o.getnnz() for o in test_dset.data.data_lbl])

In [ ]:
m = metric(**o)

/scratch/scai/phd/aiz218323/anaconda3/envs/xc_nlg/lib/python3.9/site-packages/scipy/sparse/_index.py:145: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [ ]:
with pd.option_context('display.max_columns', None):
    display(pd.DataFrame([m])[sorted_metric(m, order=['P', 'N', 'PSP', 'PSN'])] * 100)

,P@1,P@3,P@5,P@10,N@1,N@3,N@5,N@10,PSP@1,PSP@3,PSP@5,PSP@10,PSN@1,PSN@3,PSN@5,PSN@10,R@2,R@3,R@10,R@50,R@100,R@200
0,8.6,5.5,4.1,2.685,8.6,9.178565,9.570599,10.328424,6.63238,7.007277,7.571821,9.066958,6.63238,7.530207,8.075479,8.855146,7.7908,8.942542,12.572873,20.347874,25.829074,34.116347


In [ ]:
o['pred_score'] = sc

In [ ]:
m = metric(**o)

/scratch/scai/phd/aiz218323/anaconda3/envs/xc_nlg/lib/python3.9/site-packages/scipy/sparse/_index.py:145: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [ ]:
with pd.option_context('display.max_columns', None):
    display(pd.DataFrame([m])[sorted_metric(m, order=['P', 'N', 'PSP', 'PSN'])] * 100)

,P@1,P@3,P@5,P@10,N@1,N@3,N@5,N@10,PSP@1,PSP@3,PSP@5,PSP@10,PSN@1,PSN@3,PSN@5,PSN@10,R@2,R@3,R@10,R@50,R@100,R@200
0,30.15,21.533333,16.61,10.66,30.15,30.456043,31.545956,33.771454,17.870654,21.88763,24.84054,30.031887,17.870653,21.004089,22.834681,25.375879,22.681426,27.358629,39.813953,51.198876,53.720209,55.269016
